In [ ]:
ggduo
From GGally v1.2.0
by Barret Schloerke
97th
Percentile
Ggduo - A Ggplot2 Generalized Pairs Plot For Two Columns Sets Of A Data.Frame
Make a matrix of plots with a given data set with two different column sets

Usage
ggduo(data, mapping = NULL, columnsX = 1:ncol(data), columnsY = 1:ncol(data), title = "", types = list(continuous = "smooth_loess", comboVertical = "box", comboHorizontal = "facethist", discrete  = "ratio"), axisLabels = c("show", "none"), columnLabelsX = colnames(data[columnsX]), columnLabelsY = colnames(data[columnsY]), showStrips = NULL, legends = FALSE)


In [1]:
library(ggplot2)
install.packages("GGally")
library(GGally)

Installing package into '/home/nbcommon/R'
(as 'lib' is unspecified)


In [2]:
data(baseball, package = "plyr")

 # Add how many singles a player hit
 # (must do in two steps as X1b is used in calculations)
 dt <- transform(
   subset(baseball, year >= 1990 & year <= 1995),
   X1b = h - X2b - X3b - hr
 )
 # Add
 #  the player's batting average,
 #  the player's slugging percentage,
 #  and the player's on base percentage
 # Make factor a year, as each season is discrete
 dt <- transform(
   dt,
   batting_avg = h / ab,
   slug = (X1b + 2*X2b + 3*X3b + 4*hr) / ab,
   on_base = (h + bb + hbp) / (ab + bb + hbp),
   year = as.factor(year)
 )


In [11]:
pm <- ggduo(
   dt,
   c("year", "g", "ab", "lg"),
   c("batting_avg", "slug", "on_base"),
   mapping = ggplot2::aes(color = lg))
pm


 

ERROR: Error in eval(expr, envir, enclos): could not find function "ggduo"


In [12]:
# Make a fake column that will be calculated when printing
 dt$hit_type <- paste("hit_type:", seq_len(nrow(dt)))



In [6]:
display_hit_type <- function(plot_fn, is_ratio) {
   function(data, mapping, ...) {
     # change the color aesthetic to fill aesthetic
     mapping <- mapping_color_to_fill(mapping)

     # If the y varaible is not 'hit_type', continue like normal
     if (deparse(mapping$y) != "hit_type") {
       p <- plot_fn(data, mapping, ...)
       return(p)
     }

ERROR: Error in parse(text = x, srcfile = src): <text>:11:0: unexpected end of input
9:        return(p)
10:      }
   ^


In [7]:


     # Capture any extra column names needed
     extra_columns <- unname(unlist(lapply(
       mapping[! names(mapping) %in% c("x", "y")],
       deparse
     )))
     extra_columns <- extra_columns[extra_columns %in% colnames(data)]

     x_name <- deparse(mapping$x)

     # get the types of hits
     hit_types <- c("X1b", "X2b", "X3b", "hr")
     hit_names <- c("single", "double", "tripple", "home\nrun")
     if (is_ratio) {
       hit_types <- rev(hit_types)
       hit_names <- rev(hit_names)
     }

     # retrieve the columns and rename them
     data <- data[, c(x_name, hit_types, extra_columns)]
     colnames(data) <- c(x_name, hit_names, extra_columns)

     # melt the data to get the counts of the unique hit occurances
     dt_melt <- reshape::melt.data.frame(data, id = c(x_name, extra_columns))
     dt_value <- dt_melt$value

     # Make a new data.frame with all the necessary variables repeated
     dt_ratio <- data.frame(variable = logical(sum(dt_value)))
     for (col in c(x_name, "variable", extra_columns)) {
       dt_ratio[[col]] <- rep(dt_melt[[col]], dt_value)
     }

     # copy the old mapping and overwrite the x and y values
     mapping_ratio <- mapping
     mapping_ratio[c("x", "y")] <- ggplot2::aes_string(x = x_name, y = "variable")

     # make ggplot2 object!
     plot_fn(dt_ratio, mapping_ratio, ...)
   }
 }


 display_hit_type_combo <- display_hit_type(ggally_facethist, FALSE)
 display_hit_type_discrete <- display_hit_type(ggally_ratio, TRUE)

 # remove the strips, as the same information is displayed in the bottom axis area
 pm <- ggduo(
   dt,
   c("year", "g", "ab", "lg"),
   c("batting_avg", "slug", "on_base", "hit_type"),
   columnLabelsX = c("year", "player game count", "player at bat count", "league"),
   columnLabelsY = c("batting avg", "slug %", "on base %", "hit type"),
   title = "Baseball Hitting Stats from 1990-1995",
   mapping = ggplot2::aes(color = lg),
   types = list(
     # change the shape and add some transparency to the points
     continuous = wrap("smooth_loess", alpha = 0.50, shape = "+"),
     # all combinations that are continuous horizontally should have a binwidth of 15
     comboHorizontal = wrap(display_hit_type_combo, binwidth = 15),
     # the ratio plot should have a black border around the rects of size 0.15
     discrete = wrap(display_hit_type_discrete, color = "black", size = 0.15)
   ),
   showStrips = FALSE
 );

 pm







ERROR: Error in parse(text = x, srcfile = src): <text>:40:4: unexpected '}'
39:      plot_fn(dt_ratio, mapping_ratio, ...)
40:    }
       ^


In [ ]:
# Example derived from:
## R Data Analysis Examples: Canonical Correlation Analysis.  UCLA: Statistical
##   Consulting Group. from http://www.ats.ucla.edu/stat/r/dae/canonical.htm
##   (accessed June 23, 2016).


#### Example 1. 
* A researcher has collected data on three psychological variables, four academic variables (standardized test scores) and gender for 600 college freshman.
* She is interested in how the set of psychological variables relates to the academic variables and gender. 
* In particular, the researcher is interested in how many dimensions (canonical variables) are necessary to understand the association between the two sets of variables.

In [8]:

mm <- read.csv("http://www.ats.ucla.edu/stat/data/mmreg.csv")
colnames(mm) <- c("Control", "Concept", "Motivation", "Read", "Write", "Math",
    "Science", "Sex")
summary(mm)

psych_variables <- c("Control", "Concept", "Motivation")
academic_variables <- c("Read", "Write", "Math", "Science", "Sex")


Warning message in file(file, "rt"):
"cannot open URL 'http://www.ats.ucla.edu/stat/data/mmreg.csv': HTTP status was '403 Forbidden'"

ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
## Within correlation
ggpairs(mm, columns = psych_variables)


In [ ]:
ggpairs(mm, columns = academic_variables)


In [9]:

## Between correlation
loess_with_cor <- function(data, mapping, ..., method = "pearson") {
  x <- data[[deparse(mapping$x)]]
  y <- data[[deparse(mapping$y)]]
  cor <- cor(x, y, method = method)
  ggally_smooth_loess(data, mapping, ...) +
    ggplot2::geom_label(
      data = data.frame(
        x = min(x, na.rm = TRUE),
        y = max(y, na.rm = TRUE),
        lab = round(cor, digits = 3)
      ),
      mapping = ggplot2::aes(x = x, y = y, label = lab),
      hjust = 0, vjust = 1,
      size = 5, fontface = "bold"
    )
}


ERROR: Error in eval(expr, envir, enclos): could not find function "p_"


In [13]:
pm <- ggduo(mm, psych_variables, academic_variables, types = list(continuous = loess_with_cor))
pm

ERROR: Error in eval(expr, envir, enclos): could not find function "ggduo"
